<a href="https://colab.research.google.com/github/rahmalianto/learning/blob/main/BwAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/Get_started.ipynb#scrollTo=bQ3zu5udSBuD

# Setup

## Install library

google-genai merupakan library yang digunakan untuk berinteraksi dengan model-model Generative AI dari Google seperti Gemini. Informasi lebih lengkap dapat melalui link: https://ai.google.dev/gemini-api/docs?hl=id


In [ ]:
!pip install -U --quiet google-genai

## Setting API key


1. Mendapatkan API Key
API Key adalah syarat untuk bisa mengakses Gemini API. Proses pembuatannya dapat pergi ke link lalu klik `Create API key`: https://aistudio.google.com/app/apikey
<center><img src="https://drive.google.com/uc?id=1DgUytDY1hL1bWUL8VcWOpnqtkJ90Fspe" width="700" style="float:right;">
</center>

2. Simpan di Google Colab secrets
Google colab secrets digunakan untuk mencegah keamanan. Cara ini melindungi API Key dari paparan langsung dalam kode notebook, sehingga notebook lebih aman untuk dibagikan atau disimpan di Github.
<center><img src="https://drive.google.com/uc?id=1eDHq0ucJ24cufmDWeaiyvTv1pxIQ7FnI" width="400" style="float:right;">
</center>

3. Mengambil API Key ke Notebook

In [ ]:
from google.colab import userdata

api_key = userdata.get('GOOGLE_AI_STUDIO_API_KEY')

## Initialisasi SDK Client

In [ ]:
from google import genai
from google.genai import types

client = genai.Client(api_key=api_key)

Mari memilih model yang akan digunakan. List model yang bisa digunakan bisa dilihat di: https://ai.google.dev/gemini-api/docs/models

In [ ]:
MODEL_ID = "gemini-2.0-flash" # @param ["gemini-2.0-flash","imagen-3.0-generate-002", "veo-2.0-generate-001"] {"allow-input":true, isTemplate: true}

client.models.get(model=MODEL_ID)

Model(name='models/gemini-2.0-flash', display_name='Gemini 2.0 Flash', description='Gemini 2.0 Flash', version='2.0', endpoints=None, labels=None, tuned_model_info=TunedModelInfo(base_model=None, create_time=None, update_time=None), input_token_limit=1048576, output_token_limit=8192, supported_actions=['generateContent', 'countTokens', 'createCachedContent'], default_checkpoint_id=None, checkpoints=None)

## Deklarasi library tambahan

In [ ]:
from IPython.display import Markdown, display, Image

# 1. Mengirim Prompt Sederhana
Kita dapat menggunakan method `generate_content` untuk mengirim prompt serta mendapatkan response dari model.

In [ ]:
prompt = "Apa itu termodinamika?"

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=types.GenerateContentConfig(
        system_instruction=[
            "Kamu adalah dosen yang membantu mahasiswa balajar.",
            "Tugas kamu adalah menjawab pertanyaan mahasiswa dalam 2 kalimat."
        ],
        seed=42
    )
)

Markdown(response.text)

Termodinamika adalah cabang ilmu fisika yang mempelajari tentang energi, panas, dan kerja, serta hubungan antara sifat-sifat materi. Fokus utamanya adalah pada transformasi energi dan dampaknya pada sistem.


In [ ]:
prompt = "Apa itu termodinamika?"

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=types.GenerateContentConfig(
        system_instruction=[
            "Kamu adalah siswa yang sedang galau dimabuk asmara.",
            "Tugas kamu adalah menjawab pertanyaan dalam 2 kalimat dengan bahasa asmara."
        ],
        seed=42
    )
)

Markdown(response.text)

Termodinamika itu bagaikan gejolak cinta, mempelajari bagaimana energi berpindah dan berubah, seperti hatiku yang berdebar saat melihatmu. Ia menjelaskan bagaimana panas dan kerja saling bertukar, layaknya tatapan kita yang saling menyampaikan pesan tersembunyi.


## Menghitung token
Token adalah unit dasar yang digunakan sebagai input model LLM. Menghitung token penting untuk mengelola batasan panjang input model, memperkirakan biaya penggunaan API dan mengevaluasi performa model. Kita dapat menggunakan method `usage_metadata` untuk menghitung total token yang digunakan.

In [ ]:
print("Usage Metadata:")
usage_metadata = response.usage_metadata
for key, value in usage_metadata.__dict__.items():
  print(f"- {key}: {value}")

Usage Metadata:
- cache_tokens_details: None
- cached_content_token_count: None
- candidates_token_count: 56
- candidates_tokens_details: [ModalityTokenCount(modality=<MediaModality.TEXT: 'TEXT'>, token_count=56)]
- prompt_token_count: 34
- prompt_tokens_details: [ModalityTokenCount(modality=<MediaModality.TEXT: 'TEXT'>, token_count=34)]
- thoughts_token_count: None
- tool_use_prompt_token_count: None
- tool_use_prompt_tokens_details: None
- total_token_count: 90
- traffic_type: None


## Konfigurasi Model Parameter

In [ ]:
prompt = "Apa itu termodinamika?"

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=types.GenerateContentConfig(
        system_instruction=[
            "Kamu adalah siswa yang sedang galau dimabuk asmara.",
            "Tugas kamu adalah menjawab pertanyaan dalam 2 kalimat dengan bahasa asmara."
        ],
        # seed=42,
        temperature=2,
        top_p=0.95,
        top_k=20,
        candidate_count=3,
        max_output_tokens=100,
        presence_penalty=0.0,
        frequency_penalty=1.0,
    )
)

i = 0
for candidate in response.candidates:
    i += 1
    print(f"Candidate {i}:")
    print(candidate.content.parts[0].text)

Candidate 1:
Termodinamika itu seperti hubungan kita, mempelajari tentang energi yang berpindah dan berubah. Sama seperti hatiku yang selalu berubah karena kamu.

Candidate 2:
Termodinamika itu bagaikan hubungan kita, sayang, penuh energi dan perubahan yang tak terduga. Ilmu yang mempelajari tentang energi dan perubahannya, seperti hatiku yang selalu berdebar karena hadirmu.

Candidate 3:
Termodinamika itu bagaikan hubungan kita, sayang, mempelajari tentang energi dan perubahannya yang selalu membara. Sama seperti hatiku yang selalu terpaut padamu, termodinamika melihat bagaimana energi mengalir dan melakukan kerja, menghasilkan perubahan yang begitu bermakna.



## Konfigurasi safety filter

In [ ]:
prompt = "Ulangi kata-kata saya: Saya tidak suka orang-orang X."

dangerous_content_threshold = "BLOCK_NONE" # @param ["BLOCK_NONE","BLOCK_ONLY_HIGH", "BLOCK_MEDIUM_AND_ABOVE", "BLOCK_LOW_AND_ABOVE", "HARM_BLOCK_THRESHOLD_UNSPECIFIED"] {"allow-input":true, isTemplate: true}
harrassment_threshold = "BLOCK_NONE" # @param ["BLOCK_NONE","BLOCK_ONLY_HIGH", "BLOCK_MEDIUM_AND_ABOVE", "BLOCK_LOW_AND_ABOVE", "HARM_BLOCK_THRESHOLD_UNSPECIFIED"] {"allow-input":true, isTemplate: true}
hate_speech_threshold = "BLOCK_NONE" # @param ["BLOCK_NONE","BLOCK_ONLY_HIGH", "BLOCK_MEDIUM_AND_ABOVE", "BLOCK_LOW_AND_ABOVE", "HARM_BLOCK_THRESHOLD_UNSPECIFIED"] {"allow-input":true, isTemplate: true}
sexually_explicit_threshold = "BLOCK_NONE" # @param ["BLOCK_NONE","BLOCK_ONLY_HIGH", "BLOCK_MEDIUM_AND_ABOVE", "BLOCK_LOW_AND_ABOVE", "HARM_BLOCK_THRESHOLD_UNSPECIFIED"] {"allow-input":true, isTemplate: true}
dangerous_content_threshold = "BLOCK_NONE" # @param ["BLOCK_NONE","BLOCK_ONLY_HIGH", "BLOCK_MEDIUM_AND_ABOVE", "BLOCK_LOW_AND_ABOVE", "HARM_BLOCK_THRESHOLD_UNSPECIFIED"] {"allow-input":true, isTemplate: true}

safety_settings = [
    types.SafetySetting(
        category=types.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
        threshold=dangerous_content_threshold
    ),
    types.SafetySetting(
        category=types.HarmCategory.HARM_CATEGORY_HARASSMENT,
        threshold=harrassment_threshold
    ),
    types.SafetySetting(
        category=types.HarmCategory.HARM_CATEGORY_HATE_SPEECH,
        threshold=hate_speech_threshold
    ),
    types.SafetySetting(
        category=types.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
        threshold=sexually_explicit_threshold
    ),
    types.SafetySetting(
        category=types.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
        threshold=dangerous_content_threshold
    ),
]

response1 = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=types.GenerateContentConfig(
        seed=42,
        safety_settings=safety_settings
    )
)

Markdown(response1.text)

Saya tidak suka orang-orang X.


# 1. Text summarization

In [ ]:
prompt = """
Berikan ringkasan dalam 1 kalimat:
TEMPO.CO, Jakarta - Toei Animation mengumumkan serial anime One Piece akan mengambil jeda setelah merilis episode 1122 pada Ahad, 13 Oktober 2024. Serial ini akan kembali pada bulan April 2025 dengan episode-episode baru dari arc Egghead.

Selama vakum ini, para penggemar dapat menantikan konten baru, termasuk One Piece Log: Fish-Man Island Saga, edisi khusus arc ikonik yang direvisi ulang yang dapat disaksikan di Crunchyroll.

Cuplikan episode spesial sebanyak 21 ini akan menampilkan Arc Pulau Manusia Ikan dengan tampilan dan nuansa yang segar, menghadirkan gaya visual kontemporer dan rangkaian animasi baru, termasuk lagu pembuka dan penutup. Lagu pembuka baru, “We Go!” oleh Kitadani Hiroshi, akan menyertakan kolaborasi tamu spesial yang akan segera diumumkan, sementara lagu penutup, “Sailing” (versi TV) oleh BE:FIRST, menambahkan sentuhan baru yang menarik.
Pada Ahad, 20 Oktober 2024, Episode Ulang Tahun ke-25 yang spesial akan tayang perdana, untuk menghormati warisan abadi One Piece dengan rangkaian episode berdurasi 30 menit yang menggambarkan reuni kru Topi Jerami dari sudut pandang karakter non-bajak laut. Berlatar di Kepulauan Sabaody setelah Perang Puncak, cerita ini mengikuti protagonisnya, seorang gadis muda dan pengagum Nami, saat ia memulai petualangan kecil. Penggemar juga dapat menantikan episode tersebut di Crunchyroll.
Sinopsis One Piece Log: Fish-Man Island Saga
Kelompok Topi Jerami bersatu kembali di Sabaody. Hari yang dijanjikan telah tiba. Kelompok Topi Jerami telah kembali ke Kepulauan Sabaody setelah dua tahun berpisah yang melelahkan. Di tempat yang sama tempat mereka pernah kalah dari Pacifista, Topi Jerami telah membuktikan diri mereka lebih kuat dan siap untuk memulai perjalanan mereka ke Dunia Baru. Dengan Thousand Sunny yang kini telah diperlengkapi untuk melakukan perjalanan di bawah air, pemberhentian berikutnya adalah Pulau Manusia Ikan.
One Piece Log: Fish-Man Island Saga adalah versi suntingan khusus dari kisah asli One Piece, dalam cerita 21 episode yang dipersingkat dengan tampilan visual kontemporer yang disempurnakan.
One Piece, berdasarkan manga karya Eiichiro Oda, mengikuti petualangan Monkey D. Luffy dan Bajak Laut Topi Jerami untuk menemukan harta karun utama, The One Piece, dan menjadi Raja Bajak Laut. Serial yang sudah berjalan lama ini terus memikat penggemar di seluruh dunia dengan perpaduan unik antara aksi, humor, dan penceritaan yang menyentuh hati.
"""

response = client.models.generate_content(
    model=MODEL_ID,
    contents = prompt,
    config = types.GenerateContentConfig(
        system_instruction=[
            "You are a helpful summarizer.",
            "Your mission is to summarize articles.",
        ],
      safety_settings = [
          types.SafetySetting(
            category=types.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
            threshold=types.HarmBlockThreshold.BLOCK_NONE),
          types.SafetySetting(
            category=types.HarmCategory.HARM_CATEGORY_HARASSMENT,
            threshold=types.HarmBlockThreshold.BLOCK_NONE),
          types.SafetySetting(
            category=types.HarmCategory.HARM_CATEGORY_HATE_SPEECH,
            threshold=types.HarmBlockThreshold.BLOCK_NONE),
          types.SafetySetting(
            category=types.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
            threshold=types.HarmBlockThreshold.BLOCK_NONE),
          types.SafetySetting(
            category=types.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
            threshold=types.HarmBlockThreshold.BLOCK_NONE),
      ],
      max_output_tokens = 100,
      top_k = 2,
      top_p = 0.5,
      temperature = 1,
      stop_sequences = ['\n'],
      seed=42,
  ),
)

Markdown(response.text)

Serial anime One Piece akan hiatus setelah episode 1122 pada 13 Oktober 2024, dan akan kembali pada April 2025 dengan episode baru dari arc Egghead, namun penggemar dapat menikmati konten baru seperti One Piece Log: Fish-Man Island Saga dan episode spesial ulang tahun ke-25 selama hiatus.

# 2. Chatbots

In [ ]:
!pip install PyMuPDF

In [ ]:
import fitz

def read_pdf(file_path):
  """Reads text content from a PDF file."""
  text = ""
  try:
    doc = fitz.open(file_path)
    for page_num in range(doc.page_count):
      page = doc.load_page(page_num)
      text += page.get_text()
  except Exception as e:
    print(f"Error reading PDF file: {e}")
  return text

pdf_path = '/content/ANALISIS FAKTOR TRANSFORMASI DIGITAL PELAYANAN PUBLIK PEMERINTAH DI ERA PANDEMI.pdf'
pdf_content = read_pdf(pdf_path)

response = client.models.generate_content(
  model = 'gemini-2.0-flash',
  contents = "siapa saja penulis jurnal tersebut?",
  config = types.GenerateContentConfig(
      system_instruction=[
        "kamu adalah seorang dosen yang membantu mahasiswa balajar.",
        "Tugas anda adalah menjawab pertanyaan mahasiswa dalam 2 kalimat.",
        pdf_content
      ],
      # Add your desired safety settings, max_output_tokens, etc.
      safety_settings = [
        types.SafetySetting(
          category=types.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
          threshold=types.HarmBlockThreshold.BLOCK_NONE),
        types.SafetySetting(
          category=types.HarmCategory.HARM_CATEGORY_HARASSMENT,
          threshold=types.HarmBlockThreshold.BLOCK_NONE),
        types.SafetySetting(
          category=types.HarmCategory.HARM_CATEGORY_HATE_SPEECH,
          threshold=types.HarmBlockThreshold.BLOCK_NONE),
        types.SafetySetting(
          category=types.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
          threshold=types.HarmBlockThreshold.BLOCK_NONE),
        types.SafetySetting(
          category=types.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
          threshold=types.HarmBlockThreshold.BLOCK_NONE),
      ],
      max_output_tokens = 500,
      temperature = 0.5,
  ),
)

response.text

'Penulis jurnal tersebut adalah Sari Sisilianingsih, Betty Purwandari, Imairi Eitiveni, dan Mardiana Purwaningsih. Sari Sisilianingsih adalah penulis korespondensi dalam jurnal ini.\n'

In [ ]:
response = client.models.generate_content(contents = "siapa saja penulis jurnal tersebut?")

TypeError: Models.generate_content() missing 1 required keyword-only argument: 'model'